# Analyzing AAL Sentiment Using FinBERT

In this notebook, I will be analyzing the different sections of AAL's 10K reports throughout the years, and brainstorm ideas on how I will score companies and create a signal. 

To get a sentiment score, I will be using an NLP model called FinBERT. You can read more about FinBERT here: https://github.com/ProsusAI/finBERT

To fit the model, you will need to download the pre-trained and fine-tuned weights from the following link and unzip to the working directory: https://gohkust-my.sharepoint.com/:u:/g/personal/imyiyang_ust_hk/EQJGiEOkhIlBqlW63TbKA3gBCYgDDcHlBCB7VTXIUMmyiA

**Note that I did not include the analyst_tone folder in my repository, this is because it's simply too large to include, so you will need to download it yourself**

# Toy Example

First, I will be borrowing code directly from a GitHub that I had used previously to also work with FinBERT. I recall that I had some issues on that machine, so I want to run a toy example before looking at AAL. You can find exactly where I get the code from here:

https://github.com/yya518/FinBERT/blob/master/FinBert%20Model%20Example.ipynb

In [2]:
import numpy as np 
import pandas as pd

import pickle
import re

import torch
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer
from FinBERT_master.bertModel import BertClassification

In [3]:
labels = {0:0, 1:1,2:-1}
num_labels= len(labels)
vocab = "finance-uncased"
vocab_path = '../../analyst_tone/vocab'
pretrained_weights_path = "../../analyst_tone/pretrained_weights" # this is pre-trained FinBERT weights
fine_tuned_weight_path = "../../analyst_tone/fine_tuned.pth"      # this is fine-tuned FinBERT weights
max_seq_length=512
device= torch.device('cpu')

In [3]:
model = BertClassification(weight_path= pretrained_weights_path, num_labels=num_labels, vocab=vocab)

/Users/lucaskrenn/Documents/HKML/SP500_NLP/SP500_NLP_Project/Step2-Signal/FinBERT_master/bertModel.py:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.classifier.weight)


In [4]:
model.load_state_dict(torch.load(fine_tuned_weight_path, map_location='cpu'))
model.to(device)

BertClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )


In [28]:
sentences = ["there is a shortage of capital, and we need extra financing", 
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]

In [29]:
tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)

In [30]:
model.eval()
for sent in sentences: 
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')


there is a shortage of capital, and we need extra financing 
FinBERT predicted sentiment:  -1 

growth is strong and we have plenty of liquidity 
FinBERT predicted sentiment:  1 

there are doubts about our finances 
FinBERT predicted sentiment:  -1 

profits are flat 
FinBERT predicted sentiment:  0 



# Working With AAL Data

Now, I will import the pickled information from the scrape file and try to develop some way to score the sentiment of each section. Note that the parsed.pickle file **is not saved in the Github and will need to be downloaded from the all_stock_parse.ipynb notebook**

In [4]:
with open('../Step1-Data/4-link_dict.pickle', 'rb') as f:
    parsed = pickle.load(f)

In [5]:
sample = parsed['0000006201']['Item7'].values[0]

In [6]:
cleaned_sample = re.sub(r"[\n\t]", " ", sample)
cleaned_sample

'ITEM 7.  MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS  2020 Financial Overview Impact of Coronavirus (COVID-19)  COVID-19 has been declared a global health pandemic by the World Health Organization. COVID-19 has surfaced in nearly all regions of the world, which has driven  the  implementation of significant, government-imposed measures  to prevent or  reduce  its spread,  including  travel  restrictions,  testing regimes, closing of borders, “stay at home” orders and business closures. As a result, we have experienced an unprecedented decline in the demand for air travel, which has resulted in a material deterioration in our revenues. While our business performed largely as expected in January and February of 2020, a severe reduction in air travel starting in March 2020 resulted in our total operating revenues decreasing approximately 62% in 2020 as compared to 2019. While the  length and severity of  the reduction  in demand due to the COVID-

In [7]:
sentences = [x for x in cleaned_sample.split('. ') if x != '']
sentences

['ITEM 7',
 ' MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS  2020 Financial Overview Impact of Coronavirus (COVID-19)  COVID-19 has been declared a global health pandemic by the World Health Organization',
 'COVID-19 has surfaced in nearly all regions of the world, which has driven  the  implementation of significant, government-imposed measures  to prevent or  reduce  its spread,  including  travel  restrictions,  testing regimes, closing of borders, “stay at home” orders and business closures',
 'As a result, we have experienced an unprecedented decline in the demand for air travel, which has resulted in a material deterioration in our revenues',
 'While our business performed largely as expected in January and February of 2020, a severe reduction in air travel starting in March 2020 resulted in our total operating revenues decreasing approximately 62% in 2020 as compared to 2019',
 'While the  length and severity of  the reduction  in demand d

In [18]:
import time
start = time.time()
model.eval()
for sent in sentences: 
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        #print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')
end = time.time()

In [8]:
print(end-start)

NameError: name 'end' is not defined

In [10]:
len(sentences)

430

In [11]:
430 / 540 

0.7962962962962963

This seems like something we can work with. Now, I think it would make sense to create some sort of score. Perhaps by changing the mapping of the labels to -1\/0\/1 for negative/neutral/postive, then I can gauge the sentiment of the whole section based on the contents of the individual sentences. Let's see if we can get a score for the full section:

In [17]:
sample = parsed['AAL']['08']['item7']
cleaned_sample = re.sub(r"[\n\t]", " ", sample)
sentences = [x for x in cleaned_sample.split('. ') if x != '']

In [18]:
score = []

model.eval()
for sent in sentences: 
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        #print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')
        score.append(labels[torch.argmax(outputs).item()])

In [32]:
score = []
sent = cleaned_sample
sent

'ITEM 7.  MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS  2020 Financial Overview Impact of Coronavirus (COVID-19)  COVID-19 has been declared a global health pandemic by the World Health Organization. COVID-19 has surfaced in nearly all regions of the world, which has driven  the  implementation of significant, government-imposed measures  to prevent or  reduce  its spread,  including  travel  restrictions,  testing regimes, closing of borders, “stay at home” orders and business closures. As a result, we have experienced an unprecedented decline in the demand for air travel, which has resulted in a material deterioration in our revenues. While our business performed largely as expected in January and February of 2020, a severe reduction in air travel starting in March 2020 resulted in our total operating revenues decreasing approximately 62% in 2020 as compared to 2019. While the  length and severity of  the reduction  in demand due to the COVID-

In [33]:
tokenized_sent = tokenizer.tokenize(sent)
if len(tokenized_sent) > max_seq_length:
    tokenized_sent = tokenized_sent[:max_seq_length]

In [34]:
ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
mask_input = [1]*len(ids_review)        
padding = [0] * (max_seq_length - len(ids_review))
ids_review += padding
mask_input += padding
input_type = [0]*max_seq_length

In [35]:
input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)

In [37]:
with torch.set_grad_enabled(False):
    outputs = model(input_ids, token_type_ids, attention_mask)
    outputs = F.softmax(outputs,dim=1)
    print('\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')
    #score.append(labels[torch.argmax(outputs).item()])


FinBERT predicted sentiment:  -1 



In [19]:
sum(score)

25

In [20]:
np.mean(score)

0.07987220447284345

In [21]:
np.median(score)

0.0

Interesting, I will need to validate that the algorithm is working as intended (#ToDo) but for now, I will calculate the scores for each of the sections to get a better understanding of how a full document will be rated. 

In [25]:
[x for x in AAL['08'].keys()]

NameError: name 'AAL' is not defined

In [27]:
scores_dict = {}

for key in parsed['AAL']['08'].keys():
    sample = parsed['AAL']['08'][key]
    cleaned_sample = re.sub(r"[\n\t]", " ", sample)
    sentences = [x for x in cleaned_sample.split('. ') if x != '']
    
    score = []

    model.eval()
    for sent in sentences: 
        tokenized_sent = tokenizer.tokenize(sent)
        if len(tokenized_sent) > max_seq_length:
            tokenized_sent = tokenized_sent[:max_seq_length]

        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
        mask_input = [1]*len(ids_review)        
        padding = [0] * (max_seq_length - len(ids_review))
        ids_review += padding
        mask_input += padding
        input_type = [0]*max_seq_length

        input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
        attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
        token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)

        with torch.set_grad_enabled(False):
            outputs = model(input_ids, token_type_ids, attention_mask)
            outputs = F.softmax(outputs,dim=1)
            #print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')
            score.append(labels[torch.argmax(outputs).item()])
        
    scores_dict[key] = score
    print(key)

item1
item1a
item1b
item2
item3
item4
item5
item6
item7
item7(
item8
item9
item9a
item10
item11
item12
item13
item14


In [28]:
[sum(x) for x in scores_dict.values()]

[5, -66, 0, 0, -21, 0, 0, 1, 25, -2, -1, 0, 2, 0, 0, 0, 0, 0]

Wow! Item 1a has a seriously low score. Let's take a look:

In [91]:
AAL['08']['item1a'][:500]

'>ITEM 1A. RISK FACTORS  \nOur ability to become consistently profitable and our ability to continue to\nfund our obligations on an ongoing basis will depend on a number of risk\nfactors, many of which are largely beyond our control. Some of the factors\nthat may have a negative impact on us are described below:  \nAs a result of significant losses in recent years, our financial condition has\nbeen materially weakened.  \nWe incurred significant losses in recent prior years: $857 million in 2005,\n$751 m'

As these are risk factors, this is to be expected. Let's take a look at some other statistics

In [92]:
[np.mean(x) for x in scores_dict.values()]

[0.015384615384615385,
 -0.4074074074074074,
 0.0,
 0.0,
 -0.19811320754716982,
 0.0,
 0.0,
 0.0790273556231003,
 -0.0625,
 -0.002105263157894737,
 0.0,
 0.06451612903225806,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

Perhaps the mean is the best way to look at these? 